In [2]:
install.packages("BiocManager")
library(BiocManager)
BiocManager::install("EBImage")
library(EBImage)
install.packages("keras")
library(keras)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Bioconductor version 3.10 (BiocManager 1.30.10), ?BiocManager::install for help

Bioconductor version '3.10' is out-of-date; the current release version '3.11'
  is available with R version '4.0'; see https://bioconductor.org/install

Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

Installing package(s) 'EBImage'

Old packages: 'ape', 'car', 'cdata', 'choroplethr', 'chron', 'cpp11', 'dplyr',
  'effects', 'emmeans', 'ggalluvial', 'ggdendro', 'greybox', 'haplo.stats',
  'HardyWeinberg', 'Hmisc', 'landscapemetrics', 'libcoin', 'MASS', 'mlr3misc',
  'mlr3pipelines', 'pbapply', 'ps', 'psychTools', 'RcmdrMisc', 'rqdatatable',
  'rquery', 'sigr', 'SNPassoc', 'soundgen', 'StanHeaders', 'TTR', 'vroom',
  'vtreat', 'workflows', 'wrapr', 'WVPlots', 'zip', 'mgcv'

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘keras’


The following obje

In [3]:
train_label<-read.csv('../input/imagedataset/train_label.csv', stringsAsFactors=TRUE)
test_label<-read.csv('../input/imagedataset/test_label.csv', stringsAsFactors=TRUE)
pat <-c("*.jpg","*.jpeg","*.png")

train_folder <- "../input/imagedataset/train_features/train_features"
train_files <- list.files(path = train_folder, pattern = pat, full.names=TRUE)
train_images <- lapply(train_files, readImage )

test_folder <- "../input/imagedataset/test_features/test_features"
test_files <- list.files(path = test_folder, pattern = pat, full.names=TRUE)
test_images <- lapply(test_files, readImage)


In [4]:
for(i in 1:length(train_images))
    {
    train_images[[i]] <- resize(train_images[[i]],28,28)
}

for(j in 1:length(test_images))
    {
    test_images[[j]] <- resize(test_images[[j]],28,28)
}

for(k in 1:length(train_images))
    {
    train_images[[k]]<-array_reshape(train_images[[k]],c(28,28,3))
}


for(l in 1:length(test_images))
    {
    test_images[[l]]<-array_reshape(test_images[[l]],c(28,28,3))
}


trainx<-NULL
testx<-NULL


In [5]:
for(i in 1:length(train_images))
    {
    trainx<-rbind(trainx,train_images[[i]])
}

for(j in 1:length(test_images))
    {
    testx <-rbind(testx, test_images[[j]])
}

In [6]:
train_label<-to_categorical(train_label)
test_label<-to_categorical(test_label)

In [7]:
model1 <- keras_model_sequential()

In [8]:
model1 %>%
layer_dense(units=32768,activation="relu",input_shape=c(2352)) %>%
layer_dense(units=16384, activation="relu") %>%
layer_dense(units=8192, activation="relu") %>%
layer_dense(units=4096, activation="relu") %>%
layer_dense(units=2048, activation="relu") %>%
layer_dense(units=2, activation="sigmoid")

In [9]:
model1 %>% 
compile(loss='binary_crossentropy', optimizer='sgd', metrics=c('accuracy'))

In [10]:
history<-model1 %>%
fit(trainx, train_label, epochs=100, batch_size=5, validation_split=0.2)

In [11]:
model1 %>% evaluate(trainx, train_label)

$loss
[1] 0.9529992

$accuracy
[1] 0.8233216

In [14]:
model1 %>% evaluate(testx, test_label)

$loss
[1] 0.918837

$accuracy
[1] 0.8309859

In [15]:
pred<-model1 %>% predict_classes(testx)

In [19]:
test_lab<-read.csv('../input/imagedataset/test_label.csv', stringsAsFactors=TRUE)

In [34]:
table(Predicted=pred, Actual=test_lab$value)

         Actual
Predicted  0  1
        0 10  5
        1  7 49